In [40]:
import os
import sys
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from random import shuffle

In [41]:
sys.path.append('..')

In [42]:
DATA_PATH = '../TlkPersonaChatRus/'

## Dialogs

In [43]:
dialogs = pd.read_csv(os.path.join(DATA_PATH, 'new_dialogues.tsv'), sep='\t', index_col=0)

In [44]:
dialogs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10013 entries, 0 to 10012
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dialogue             10013 non-null  object
 1   persona_1_profile_1  10013 non-null  object
 2   persona_1_profile_2  10013 non-null  object
 3   persona_1_profile_3  10013 non-null  object
 4   persona_1_profile_4  10013 non-null  object
 5   persona_1_profile_5  10013 non-null  object
 6   persona_2_profile_1  10013 non-null  object
 7   persona_2_profile_2  10013 non-null  object
 8   persona_2_profile_3  10013 non-null  object
 9   persona_2_profile_4  10013 non-null  object
 10  persona_2_profile_5  10013 non-null  object
dtypes: object(11)
memory usage: 938.7+ KB


### dialogs

In [45]:
some_text = dialogs['dialogue'][1]
some_text

'<span class=participant_1>Пользователь 1: Привет!</span><br /><span class=participant_2>Пользователь 2: Привет,Как жизнь?</span><br /><span class=participant_1>Пользователь 1: Отлично) Солнышко светит, птички поют!</span><br /><span class=participant_2>Пользователь 2: Я вот сегодня понял, что меня тупо используют, всем<br />нужны от меня лишь деньги, ненавижу людей</span><br /><span class=participant_2>Пользователь 2: Чем занимаешься по жизни, я вот бизнесмен.</span><br /><span class=participant_1>Пользователь 1: А я вот учу детей, работаю с начальными классами</span><br /><span class=participant_1>Пользователь 1: Не все люди такие, как ты говоришь</span><br /><span class=participant_1>Пользователь 1: Помимо работы чем еще ты занимаешься?</span><br /><span class=participant_2>Пользователь 2: К свадьбе готовлюсь</span><br /><span class=participant_2>Пользователь 2: А ты?</span><br /><span class=participant_1>Пользователь 1: Вот видишь) значит, нашел такую женщину, которой не<br />нужны

In [46]:
from preprocessing import Dialog

some replicas are corrupted by `<br />` symbol - need to fix this

In [47]:
def print_dialog(row):
    text = row['dialogue'].replace('<br />', ' ')
    print('\n'.join(t.text for t in BeautifulSoup(text, features='html').find_all('span')))
    print('\n' + '#'*80 + '\n')

In [48]:
dialog_class = Dialog(dialogs['dialogue'].tolist())
dialog_class

<speaker2>Привет, как жизнь?
<speaker1>Привет!Отлично!Сегодня замечательно прошёл день!!!
<speaker2>Хм. А я вот весь день залипал в компе. Недавно приобрёл новый. Наслаждаюсь<speaker2>Если долго отвечаю, то извини. По личным обстоятельствам живу сейчас на даче. Тут с интернетом беда
<speaker1>Наверное это интересно, но я предпочитаю книги
<speaker2>А какой любимый писатель? Моя бывшая жена, с которой как никак были вместе 40 лет, обожала паланика. От того терпеть его теперь не могу
<speaker1>Ничего у меня тоже иногда интернет тормозит. Я учитель и интернет мне очень нужен как ты понимаешь, но часто подводит! Так что я тебя понимаю.
<speaker2>Интересно наверное учителем работать. Постоянно с детьми. У меня у самого из трое. Хотел бы проводить с ними времени больше. Но работа мешает. Я сторож , график тяжёлый
<speaker1>Ну сейчас например читаю Достоевского. А дети и муж тоже больше ко пьютер предпочитают.<speaker1>Ох я тебя понимаю. У самой совершенно не хватает времени на семью с этой р

as in usual dialog there can be some consequtive replicas from one user

# IDEA - combine one type of personality and train on this

## Profiles

In [49]:
profiles = pd.read_csv(os.path.join(DATA_PATH, 'profiles.tsv'), sep='\t', )

In [50]:
profiles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1505 entries, У меня любимая работа. to Я кулинар.
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   characteristic_1  1505 non-null   object 
 1   characteristic_2  1505 non-null   object 
 2   characteristic_3  1505 non-null   object 
 3   characteristic_4  1505 non-null   object 
 4   characteristic_5  0 non-null      float64
dtypes: float64(1), object(4)
memory usage: 70.5+ KB


In [51]:
profiles.head(7)

,characteristic_1,characteristic_2,characteristic_3,characteristic_4,characteristic_5
У меня любимая работа.,Я уважаю людей.,У меня есть животное.,У меня хороший друг.,Я люблю кофе.,NaN
Я работаю учителем,У меня есть собака,Я люблю петь,Я живу сама,Я люблю цветы,NaN
Я купила дом,Я бегаю по утрам,Я работаю на работе,Я поеду в отпуск,Я люблю арбуз,NaN
я врач и женат,у меня трое детей,не люблю свою работу,нравиться ездить на велосипеде,люблю пиво,NaN
Я школьница.,Я ещё учусь.,Но я мечтаю работать.,Я обожаю родителей.,И не люблю учиться.,NaN
Я дизайнер.,Я был в 12-ти странах.,Люблю пешие прогулки.,люблю рок-музыку.,Хочу купить дачу.,NaN
я норвежец,у меня есть попугай,я не люблю деревья,у меня было десять лошадей,я не курю,NaN


In [52]:
from itertools import chain

dialogs['persona_1'] = dialogs[[f'persona_1_profile_{i}' for i in range(1,6)]].values.tolist()
dialogs['persona_2'] = dialogs[[f'persona_2_profile_{i}' for i in range(1,6)]].values.tolist()

In [53]:
def preprocess_for_raw_text(dialog):
    dialog = dialog.replace('<br />', ' ').replace('</span> ', '')
    dialog = dialog.replace('<span class=participant_1>Пользователь 1: ', '<speaker1>')
    dialog = dialog.replace('<span class=participant_2>Пользователь 2: ', '<speaker2>')
    return dialog

In [54]:
dialogs['dialogue_preprocessed'] = dialogs['dialogue'].apply(dialog_class._preprocess_for_raw_text)

In [55]:
dialogs[['dialogue_preprocessed', 'persona_1', 'persona_2']]

,dialogue_preprocessed,persona_1,persona_2
0,<speaker2>Привет) расскажи о себе<speaker1>При...,"[любимая работа, уважаю людей, есть животное, ...","[ищу принца, веду активный образ жизни, люблю ..."
1,"<speaker1>Привет!<speaker2>Привет,Как жизнь?<s...","[работаю учителем, есть собака, люблю петь, жи...","[бизнесмен, скоро свадьба, меня любят только з..."
2,<speaker1>Привет<speaker1>Как дела ?<speaker2>...,"[купила дом, бегаю по утрам, работаю на работе...","[пою в караоке, есть супруга, хорошо готовлю п..."
3,<speaker2>Здравствуйте<speaker2>Я Леша<speaker...,"[врач и женат, трое детей, не люблю свою работ...","[мальчик, учусь в 6-ом классе, хочу стать гонщ..."
4,<speaker1>Привет!<speaker2>Привет!<speaker2>Ка...,"[школьница, ещё учусь, но я мечтаю работать, о...","[простоват, люди избегают меня, быстро бегаю, ..."
...,...,...,...
10008,"""<speaker1>Привет<speaker2>Привет, как настрое...","[6 собак, не люблю кошек, работаю кинологом, л...","[музыкант, люблю веселые компании, увлекаюсь о..."
10009,<speaker1>Привет)<speaker1>Как жизнь? Чем увле...,"[верная, мне нравится уют , люблю посиделки у ...","[люблю рок, пишу статьи для журнала, езжу на в..."
10010,<speaker1>Привет<speaker2>приветствую<speaker1...,"[студент, учусь на хирурга, очень люблю комеди...","[директор турфирмы, люблю детей, катаюсь на сн..."
10011,<speaker1>Приветик)<speaker2>Привет. Меня зову...,"[моя мама живет со мной, люблю есть пироги, ес...","[воспитатель, люблю готовить, люблю велоспорт,..."


In [56]:
dialog = dialog_class.form_raw_text(dialogs['dialogue_preprocessed'][0])

result = list()

for i in range(2, len(dialog)):
    if i <= 4:
        result.append(dialog[:i])
    else:
        result.append(dialog[i - 4: i])
result

[['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )'],
 ['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться'],
 ['<speaker2>Привет) расскажи о себе',
  '<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться',
  '<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю'],
 ['<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )',
  '<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться',
  '<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю',
  '<speaker2>А я выращиваю фиалки<speaker2>И веду здоровый и активный образ жизни!'],
 ['<speaker2>Что читаешь? Мне нравится классика<speaker2>Я 

In [57]:
dialog_class.get_chunks(dialog, use_history=False)

[['<bos>',
  '<speaker2>Привет) расскажи о себе<speaker1>Привет) под вкусный кофеек настроение поболтать появилось )<speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю<speaker2>А я выращиваю фиалки<speaker2>И веду здоровый и активный образ жизни!<speaker1>Ух ты, интересно.<speaker2>Ты случайно не принц на белом коне? Я его очень жду ..<speaker1>А у меня из хобби каждую неделю тусить с моим лучшим другом)',
  '<eos>']]

In [58]:
res = [''.join(dialog) for dialog in dialog_class.form_data_to_train(use_history=True)]

  0%|          | 0/10013 [00:00<?, ?it/s]

In [59]:
res[4]

'<bos><speaker2>Что читаешь? Мне нравится классика<speaker2>Я тоже люблю пообщаться<speaker1>Люблю животных, просто обожаю, как и свою работу)<speaker1>Я фантастику люблю<speaker2>А я выращиваю фиалки<speaker2>И веду здоровый и активный образ жизни!<speaker1>Ух ты, интересно.<eos>'

In [60]:
shuffle(res)

train = res[:int(len(res) * 0.85)]
valid = res[int(len(res) * 0.85):]

with open(os.path.join(DATA_PATH, 'train_hist.txt'), 'w') as f:
    f.write('\n'.join(train))

with open(os.path.join(DATA_PATH, 'valid_hist.txt'), 'w') as f:
    f.write('\n'.join(valid))

In [ ]:
shuffle(res)

train = res[:int(len(res) * 0.85)]
valid = res[int(len(res) * 0.85):]

with open(os.path.join(DATA_PATH, 'train_pers+hist.txt'), 'w') as f:
    f.write('\n'.join(train))

with open(os.path.join(DATA_PATH, 'valid_pers+hist.txt'), 'w') as f:
    f.write('\n'.join(valid))